# Moving and Rotating a Molecular Cluster:
**Task:** 
1. Move/rotate a Molecule/Atom one by one systematically
2. Move/Rotate randomly

## Water molecule ($H_{2}O$):

Cartesian coordinates [Angstrom]:

    
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000


> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [2]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# importing de Molecule Class
from src.base_molecule import Molecule

In [5]:
# creating a Molecular object. You can use a list, a dictionary (the key MUST be "atoms")
# or another Molecule object (see notebook: 01)

water_molecule=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]

water = Molecule(water_molecule)

print(water)


	3
--system of 1 molecules and 3 total individual atoms--
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000



In [6]:
# Let's add two more water molecules using the same cartesian coordinates
water2 = water.add_fragments(water_molecule)
print(water2)

	6
--system of 2 molecules and 6 total individual atoms--
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000



In [88]:
# let's create a cluster with five (5) water molecules and one single atom (Iodine)
iodine =[("I", 0, 0, 0)]

w5_I = Molecule(
    iodine,
    water,
    water,
    water,
    water,
    water,
)

print(w5_I)

	16
--system of 6 molecules and 16 total individual atoms--
I	 0.00000000	 0.00000000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000



In [89]:
# asking for its properties
print(f"Number of individual molecules/atoms: {w5_I.total_fragments}")
print(f"Total number of atoms: {w5_I._total_atoms}")
print(f"Atomic symbols: {w5_I.symbols}")
print(f"Individual atomic masses: {w5_I.atomic_masses}")
print(f"Molecular mass: {w5_I.total_mass}")
print(f"Coordinates: {w5_I.coordinates}")
print(f"Cartesian coordinates: {w5_I.cartesian_coordinates}")

Number of individual molecules/atoms: 6
Total number of atoms: 16
Atomic symbols: ['I', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H', 'O', 'H', 'H']
Individual atomic masses: [126.9045, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008, 15.999, 1.008, 1.008]
Molecular mass: 216.9795000000001
Coordinates: [('I', 0, 0, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0), ('O', 0, 0, 0), ('H', 0.58708, 0.75754, 0), ('H', -0.58708, 0.75754, 0)]
Cartesian coordinates: [(0, 0, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754, 0), (0, 0, 0), (0.58708, 0.75754, 0), (-0.58708, 0.75754

In [90]:
# let's MOVE 2 units on xy axes the first molecule (I) and rotate 180 deg around z-axis the second molecule (water)
w5_I_moved_rotated = w5_I.translate(0, x=2, y=2, z=0).rotate(1, x=0, y=0, z=180)

print(w5_I_moved_rotated)

	16
--system of 6 molecules and 16 total individual atoms--
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
O	 0.00000000	 0.00000000	 0.00000000
H	 0.58708000	 0.75754000	 0.00000000
H	-0.58708000	 0.75754000	 0.00000000
I	 2.00000000	 2.00000000	 0.00000000
O	-0.00000000	 0.16954767	 0.00000000
H	-0.58708000	-0.58799233	 0.00000000
H	 0.58708000	-0.58799233	 0.00000000



### IMPORTANT FACTS:
- After translating a molecule, a new Molecule is created where the moved molecule appended at the end
- The same is true for a rotation
- Rotation are around the center of mass of the target molecule

In [114]:
# Finally, let's move randomly inside a sphere
from copy import deepcopy
from numpy import random

sphere_radius = 2

total_steps = 1000

random_gen = random.default_rng(1234)

# creating a brand new copy 
w5_I_new = deepcopy(w5_I)

for _ in range(total_steps):
    # molecule to be selected between [0, w.total_fragments]
    mol = random_gen.choice(w5_I.total_fragments)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-move, +move]
    move = sphere_radius / 2
    tx = move * (random_gen.uniform() - 0.5)
    ty = move * (random_gen.uniform() - 0.5)
    tz = move * (random_gen.uniform() - 0.5)

    w5_I_new = w5_I_new.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )

print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5_I_new)


 *** JOB DONE ***
after 1000 steps

	16
--system of 6 molecules and 16 total individual atoms--
I	-5.34744920	-1.52244981	 0.99065117
O	 2.50582824	-0.08464627	 3.95726595
H	 3.35687976	-0.10993486	 3.51726415
H	 2.49647105	-0.90722524	 3.46552816
O	 0.70451787	 0.54655887	-4.41482351
H	 0.47303124	-0.33126927	-4.10764037
H	 1.51883192	 0.04608982	-4.48520199
O	-1.22488128	-1.37036284	-0.94050135
H	-1.69758114	-1.00365332	-0.19176325
H	-2.15011516	-1.46966561	-1.16987225
O	 2.38708146	 2.54252751	-5.47670601
H	 1.89350508	 3.14927662	-4.92283992
H	 2.51783519	 3.39957676	-5.88523976
O	 5.10029648	-1.67762483	 3.64078372
H	 5.06159267	-0.80276307	 3.25136277
H	 4.88697565	-1.09864226	 4.37413438



In [121]:
# let's visualize its random process FREEZING Iodine
from copy import deepcopy
from numpy import random

xyz_view = py3Dmol.view(width=300,height=200)

sphere_radius = 2

total_steps = 1000

random_gen = random.default_rng(1234)

# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

w5_xyz = ""

for i in range(total_steps):
    # molecule to be selected between [0, w.total_fragments]
    mol = random_gen.choice(
        w5_I.total_fragments,
        # probability of beeing selected. FREEZING Iodine
        p=[0, 0.2, 0.2, 0.2, 0.2, 0.2], 
    )

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-move, +move]
    move = sphere_radius #/ 2
    tx = move * (random_gen.uniform() - 0.5)
    ty = move * (random_gen.uniform() - 0.5)
    tz = move * (random_gen.uniform() - 0.5)

    # saving coordinates as a string 
    w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )

print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5)


 *** JOB DONE ***
after 1000 steps

	16
--system of 6 molecules and 16 total individual atoms--
I	 0.00000000	 0.00000000	 0.00000000
O	 2.32722731	-9.86827290	 9.81925315
H	 2.60498142	-9.40529142	 10.61110578
H	 2.71939802	-10.54847643	 10.36884852
O	-9.17557816	 7.38238519	 6.39210331
H	-9.35026019	 7.14162097	 5.48103330
H	-8.94543979	 6.45338238	 6.34190483
O	 3.13384494	-7.19670176	-11.63552288
H	 3.53682624	-7.61804274	-10.87486026
H	 3.14622766	-6.51424870	-10.96274189
O	-6.49263362	-9.83669046	 3.96957474
H	-6.36660048	-10.46214116	 4.68473686
H	-5.63734441	-10.23179946	 3.79379353
O	-2.25279350	-7.81573298	 1.70304110
H	-2.43764602	-8.11043633	 2.59607484
H	-2.33655760	-7.00637356	 2.20945069



!pip install py3Dmol
import py3Dmol

In [ ]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [123]:
xyz_view = py3Dmol.view(width=1000,height=700)
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
    'radius': sphere_radius * 8, 
    'color' :'yellow',
    'alpha': 0.5,
    })
# xyz_view.addBox({'center': {'x':0, 'y':0, 'z':0},
#             'dimensions': {'w': box_size * 10, 'h': box_size * 10, 'd': box_size * 10},
#             'color':'yellow',
#             'alpha': 0.5,
#             })
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})
xyz_view.zoomTo()
xyz_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol